In [5]:
# Install necessary libraries
!pip install transformers datasets torch scikit-learn

import torch
import json
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from torch.cuda.amp import autocast, GradScaler

# Load your dataset
with open('/content/dataset_pairs.json', 'r') as file:
    data = json.load(file)

# Extract buggy and non-buggy code pairs
codes = []
labels = []

for item in data:
    for key in item.keys():
        if 'buggy' in item[key] and 'fixed' in item[key]:
            codes.append(item[key]['buggy'])
            labels.append(1)  # Label 1 for buggy code
            codes.append(item[key]['fixed'])
            labels.append(0)  # Label 0 for non-buggy code

# Train-test split
train_codes, test_codes, train_labels, test_labels = train_test_split(codes, labels, test_size=0.2, random_state=42)

# Load the CodeBERT tokenizer
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

# Define a dataset class for CodeBERT with reduced sequence length
class CodeDataset(Dataset):
    def __init__(self, codes, labels, tokenizer, max_length=256):  # Reduced max_length
        self.codes = codes
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.codes)

    def __getitem__(self, idx):
        code = self.codes[idx]
        label = self.labels[idx]

        # Tokenize the code using CodeBERT tokenizer
        tokens = self.tokenizer(
            code,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        input_ids = tokens["input_ids"].squeeze()
        attention_mask = tokens["attention_mask"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": torch.tensor(label, dtype=torch.long)
        }

# Create datasets and data loaders with smaller batch sizes
train_dataset = CodeDataset(train_codes, train_labels, tokenizer)
test_dataset = CodeDataset(test_codes, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)  # Reduced batch size
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Load the CodeBERT model for binary classification
model = RobertaForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=2)

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Use mixed precision
scaler = GradScaler()

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Gradient accumulation steps
accumulation_steps = 4


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkp

In [6]:
# Training function with mixed precision and gradient accumulation
def train_model(model, train_loader, optimizer, device, num_epochs=3, accumulation_steps=4):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        optimizer.zero_grad()  # Reset gradients

        for i, batch in enumerate(train_loader):
            # Move the batch to the device
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Use mixed precision with autocast
            with autocast():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss / accumulation_steps  # Scale the loss for gradient accumulation

            # Backward pass with scaled loss
            scaler.scale(loss).backward()

            # Accumulate gradients every `accumulation_steps`
            if (i + 1) % accumulation_steps == 0:
                # Step the optimizer
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            total_loss += loss.item() * accumulation_steps

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")

# Training the model
train_model(model, train_loader, optimizer, device, num_epochs=3)

<ipython-input-6-ed74446cc0b0>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch 1/3, Loss: 0.7003
Epoch 2/3, Loss: 0.6589
Epoch 3/3, Loss: 0.5962


In [7]:
# Evaluation function
def evaluate_model(model, test_loader, device):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Get predictions
            predictions = torch.argmax(logits, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predictions.cpu().numpy())

    # Calculate accuracy and confusion matrix
    accuracy = accuracy_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Confusion Matrix:\n{conf_matrix}')

# Evaluate the model
evaluate_model(model, test_loader, device)

Accuracy: 0.6726
Confusion Matrix:
[[37 12]
 [25 39]]
